In [4]:
from datetime import date
import math
from pathlib import Path

# Configuration variables
g_keywords = ["chinese", "japanese", "korean", "okinawan", "taiwanese", "tibetan", "\"east+asian\"", "oriental", "chinaman", "chinamen", "jap", "chink", "coolie", "celestial"]
g_search_date_range_start = date(1850, 1, 1)
g_search_date_range_end = date(2024, 7, 9)
g_num_articles_per_time_period = 20
g_time_block_range_years = 5

g_search_date_range_days = (g_search_date_range_end - g_search_date_range_start).days
g_num_time_blocks = math.ceil(g_search_date_range_days / (365 * g_time_block_range_years))

print("Number of time blocks: " + str(g_num_time_blocks))
print("Number of articles to download: " + str(g_num_time_blocks * g_num_articles_per_time_period))

def create_directories_if_do_not_exist(directories):
    for directory in directories:
        directory_path = Path.cwd() / directory
        if not directory_path.exists():
            directory_path.mkdir()
            print("Created directory: \"" + directory + "\"")

g_storage_directories = ["articles", "contexts", "uncorrected_contexts", "results"]
create_directories_if_do_not_exist(g_storage_directories)

g_time_block_number = int(input("Pick time block, 0-" + str(g_num_time_blocks - 1) + ": "))
if g_time_block_number < 0 or g_time_block_number >= g_num_time_blocks:
    print("Invalid time block!")
    exit()


def get_starting_date(time_block_num, search_date_range_start, time_block_range_years):
    return date(search_date_range_start.year + time_block_num * time_block_range_years, 1, 1)


def get_ending_date(time_block_num, search_date_range_start, search_date_range_end, time_block_range_years):
    ending_date = date(search_date_range_start.year + (time_block_num + 1) * time_block_range_years - 1, 12, 31)
    if ending_date > search_date_range_end:
        return search_date_range_end
    else:
        return ending_date


g_start_date = get_starting_date(g_time_block_number, g_search_date_range_start, g_time_block_range_years)
g_end_date = get_ending_date(g_time_block_number, g_search_date_range_start, g_search_date_range_end, g_time_block_range_years)

print("Start date: " + g_start_date.strftime('%Y-%m-%d'))
print("End date: " + g_end_date.strftime('%Y-%m-%d'))
print("Length of time period in days: " + str((g_end_date - g_start_date).days))

Number of time blocks: 35
Number of articles to download: 700
Start date: 1850-01-01
End date: 1854-12-31
Length of time period in days: 1825


Load functions

In [5]:
import random
import webbrowser
from PIL import Image
from pytesseract import pytesseract
import json
import cv2


def parse_time_block_str(time_blocks_str):
    unprocessed_time_block_str_list = time_blocks_str.split(", ")
    time_block_list = []
    for time_block_str in unprocessed_time_block_str_list:
        if "-" in time_block_str:
            time_block_range = time_block_str.split("-")
            time_block_list.extend(range(int(time_block_range[0]), int(time_block_range[1]) + 1))
        else:
            time_block_list.append(int(time_block_str))
    return time_block_list


def get_time_block_number(year, search_date_range_start, time_block_range_years):
    return math.floor((year - search_date_range_start.year) / time_block_range_years)


def get_keywords_with_num_hits(keywords, start_date, end_date):
    keywords_with_num_hits = {}
    for keyword in keywords:
        webbrowser.open("https://www.newspapers.com/search/results/?country=us&date-end=" + end_date.strftime('%Y-%m-%d') + "&date-start=" + start_date.strftime('%Y-%m-%d') + "&entity-types=page&keyword=" + keyword)
        keywords_with_num_hits[keyword] = int(input("Number of hits: ").replace(",", ""))
    return keywords_with_num_hits


def get_keywords_with_num_articles(keywords_with_num_hits, num_articles_per_time_period):
    total_hits = sum(keywords_with_num_hits.values())
    article_remainders = []
    keywords_with_num_articles = {}
    for keyword, hits in keywords_with_num_hits.items():
        keywords_with_num_articles[keyword] = int((hits / total_hits) * num_articles_per_time_period)
        article_remainders.append((keyword, ((hits / total_hits) * num_articles_per_time_period) % 1))
    article_remainders.sort(key=lambda keyword_with_article_remainder: keyword_with_article_remainder[1], reverse=True)
    num_searches_short = num_articles_per_time_period - sum(keywords_with_num_articles.values())
    for i in range(num_searches_short):
        keywords_with_num_articles[article_remainders[i][0]] += 1
    return keywords_with_num_articles


def get_keywords_with_urls_and_num_articles(keywords_with_num_articles, start_date, end_date):
    keywords_with_urls_and_num_articles = []
    for keyword, num_articles in keywords_with_num_articles.items():
        keywords_with_urls_and_num_articles.append({
            "keyword": keyword,
            "url": "https://www.newspapers.com/search/results/?country=us&date-end=" + end_date.strftime('%Y-%m-%d') + "&date-start=" + start_date.strftime('%Y-%m-%d') + "&entity-types=page&keyword=" + keyword,
            "num_articles": num_articles
        })
    return keywords_with_urls_and_num_articles


def run_article_download_helper(keywords_with_urls_and_num_articles):
    for keyword_with_urls_and_num_articles in keywords_with_urls_and_num_articles:
        if keyword_with_urls_and_num_articles["num_articles"] != 0:
            text_input = input("Please download " + str(keyword_with_urls_and_num_articles["num_articles"]) + " articles with the keyword \"" + keyword_with_urls_and_num_articles["keyword"] + "\". Press return to continue. Type \"stop\" to stop.")
            if text_input == "stop":
                break
            webbrowser.open(keyword_with_urls_and_num_articles["url"])
            text_input = input("Once you have downloaded " + str(keyword_with_urls_and_num_articles["num_articles"]) + " articles with the keyword \"" + keyword_with_urls_and_num_articles["keyword"] + "\", press return to continue. Type \"stop\" to stop.")
            if text_input == "stop":
                break



def get_info_from_file_stem(file_stem):
    parts = file_stem.split('_')
    newspaper_name_parts = []

    for part in parts:
        if part.isdigit() and len(part) == 4:
            year_index = parts.index(part)
            break
        newspaper_name_parts.append(part)

    newspaper_name = ' '.join(newspaper_name_parts)
    date_published = date(int(parts[year_index]), int(parts[year_index + 1]), int(parts[year_index + 2]))

    return newspaper_name, date_published


def write_time_period_sorted_info_to_files(contexts, target_dir_path):
    if not target_dir_path.exists():
        target_dir_path.mkdir()
    for time_period, time_period_info in contexts.items():
        with open(target_dir_path / f"{time_period}.json", 'w+') as time_period_info_file:
            json.dump(time_period_info, time_period_info_file, indent=4)

Get the number of articles needed for each keyword by pasting in the number of hits for each keyword

In [24]:
g_keywords_with_num_articles = get_keywords_with_num_articles(get_keywords_with_num_hits(g_keywords, g_start_date, g_end_date), g_num_articles_per_time_period)
print(g_keywords_with_num_articles)

{'chinese': 9, 'japanese': 8, 'korean': 0, 'okinawan': 0, 'taiwanese': 0, 'tibetan': 0, '"east+asian"': 0, 'oriental': 0, 'chinaman': 2, 'chinamen': 1, 'jap': 0, 'chink': 0, 'coolie': 0, 'celestial': 0}


Generate the URLs to grab the articles from. Note: number of URLs is not necessarily equal to number of articles to download.

In [25]:
g_keywords_with_urls_and_num_articles = get_keywords_with_urls_and_num_articles(g_keywords_with_num_articles, g_start_date, g_end_date)
print("Generated " + str(len(g_keywords_with_urls_and_num_articles)) + " URLs")

Generated 14 URLs


Utility to guide users in downloading articles. Place articles in a folder with the same name as the keyword, which will be in the "articles" folder. Name the file with the date in the format "YYYY-MM-DD"

In [26]:
run_article_download_helper(g_keywords_with_urls_and_num_articles)

Load NLP

In [6]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import numpy as np
import re

nlp_sentence_segmentation = spacy.load("en_core_web_trf")
nlp_sentiment_analysis = spacy.load("en_core_web_trf")
nlp_sentiment_analysis.add_pipe("spacytextblob")


def get_all_contexts_for_keywords(articles_dir_path, time_period_list):
    keywords_with_contexts_by_time_period = {}
    for time_period in time_period_list:
        keywords_with_contexts_by_time_period[time_period] = {}
        time_period_dir_path = articles_dir_path / str(time_period)
        for articles_by_keyword_dir_path in time_period_dir_path.rglob("*"):
            if articles_by_keyword_dir_path.is_dir():
                keyword = articles_by_keyword_dir_path.name
                keywords_with_contexts_by_time_period[time_period][keyword] = []
                for article_path in articles_by_keyword_dir_path.rglob("*"):
                    if article_path.is_file():
                        newspaper_name, date = get_info_from_file_stem(article_path.stem)
                        img = cv2.imread(article_path, cv2.IMREAD_GRAYSCALE)
                        blur = cv2.GaussianBlur(img,(5, 5), 0)
                        th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                        text = pytesseract.image_to_string(th3)
                        cleaned_text = cleaned_text = re.sub(r'\s+', ' ', text.replace(" |", "").replace("|", "").replace("-\n", "").replace("\n", " "))
                        keywords_with_contexts_by_time_period[time_period][keyword].append({
                            "newspaper": newspaper_name,
                            "date": date.strftime('%Y-%m-%d'),
                            "keyword": keyword,
                            "text": cleaned_text
                        })


def run_analysis_from_contexts(time_period_list):
    keywords_with_contexts_and_analysis_by_time_period = {}
    for time_block_number in time_period_list:
        keywords_with_contexts_and_analysis_by_time_period[time_block_number] = {}
        with open(Path.cwd() / "contexts" / f"{time_block_number}.json") as contexts_file:
            keywords_with_contexts = json.load(contexts_file)
            for keyword, contexts in keywords_with_contexts.items():
                keywords_with_contexts_and_analysis_by_time_period[time_block_number][keyword] = []
                for context in contexts:
                    doc = nlp_sentiment_analysis(context['text'])
                    context['polarity'] = doc._.blob.polarity
                    context['subjectivity'] = doc._.blob.subjectivity
                    keywords_with_contexts_and_analysis_by_time_period[time_block_number][keyword].append(context)
    return keywords_with_contexts_and_analysis_by_time_period

Get contexts surrounding keywords with OCR and sentence segmentation. Provide a list of numbers (e.g. 1, 2, 3) and/or a range (e.g. 10-15). Write to file in all_contexts

In [44]:
g_articles_dir_path = Path.cwd() / 'articles'
write_time_period_sorted_info_to_files(get_all_contexts_for_keywords(g_articles_dir_path, parse_time_block_str(input("Enter time periods. Provide a list of numbers (e.g. 1, 2, 3) and/or a range (e.g. 10-15)."))), Path.cwd() / 'uncorrected_contexts')

python(26491) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26493) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26494) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26495) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26496) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26497) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26498) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26499) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26500) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26501) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26502) Malloc

Read in contexts from JSON file for analysis, perform and output to results

In [8]:
write_time_period_sorted_info_to_files(run_analysis_from_contexts(parse_time_block_str(input("Enter time periods for sentiment analysis. Provide a list of numbers (e.g. 1, 2, 3) and/or a range (e.g. 10-15)."))), Path.cwd() / 'results')